In [ ]:
import nltk
nltk.download('punkt')
import torch
import numpy as np
import datasets
from datasets import load_dataset, load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    AutoConfig
)

In [ ]:
# WARNING: long process (xsum, cnn)
#reddit = load_dataset("reddit", cache_dir="../cache")
#cnn = load_dataset("cnn_dailymail", "3.0.0", cache_dir="../cache")
#reddit_tifu = load_dataset("reddit_tifu", "long", cache_dir="../cache")
#samsum = load_dataset("samsum", cache_dir="../cache")
#cnn.save_to_disk("../data/cnn-dm")
#reddit_tifu.save_to_disk("../data/reddit-tifu")
#samsum.save_to_disk("../data/samsum")

In [ ]:
samsum = load_from_disk("../data/samsum")

model_name = "facebook/bart-large"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir = "./cache")

samsum

In [ ]:
max_source_length = 512
max_target_length = 64

def preprocess_function(examples):
    inputs = [doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=max_source_length, truncation=True, return_length=True, return_overflowing_tokens=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True, return_length=True, return_overflowing_tokens=True)

    model_inputs["labels"] = labels["input_ids"]

    print(model_inputs)
    print(max(labels.length))
    print(max(model_inputs.length))
    print(max(model_inputs.overflowing_tokens))
    print(max(model_inputs.num_truncated_tokens))
    
    return model_inputs

test = samsum["test"].map(preprocess_function, batched=True)

len(test)

In [ ]:
print(len(test["input_ids"]))
print(len(test["labels"]))

In [ ]:
# reddit
# features: ['author', 'subreddit', 'summary', 'text']
# num_rows: 3848330
#reddit = reddit["train"].remove_columns(["id", "subreddit_id", "body", "normalizedBody"])
#reddit = reddit.rename_column("content", "text")
#reddit.save_to_disk("../data/reddit")
#reddit = load_from_disk("../data/reddit")
#reddit

In [ ]:
# reddit-dota
# features: ['author', 'summary', 'text']
# num_rows: 22366
#dota = reddit.filter(lambda x: x["subreddit"] == "DotA2")
#dota = dota.remove_columns("subreddit")

# bad text/summary @ dota[9]
#dota = dota.filter(lambda x: len(x["text"]) > len(x["summary"])) # source length > target length
#dota.flatten_indices().save_to_disk("../data/reddit-dota")
dota = load_from_disk("../data/reddit-dota")
dota

In [ ]:
# reddit-dota-proc
# num_rows: 19949
#max_text_length = 50000
min_summary_length = 20
max_ratio = 0.5
min_ratio = 0.005
#dota = dota.filter(lambda x: len(x["text"]) <= max_text_length)
proc = dota.filter(lambda x: len(x["summary"]) >= min_summary_length)
proc = proc.filter(lambda x: len(x["summary"])/len(x["text"]) <= max_ratio)
proc = proc.filter(lambda x: len(x["summary"])/len(x["text"]) >= min_ratio)
# bertscore filter?
proc.flatten_indices().save_to_disk("../data/reddit-dota-proc")
proc

In [ ]:
# reddit-dota-qtr
# num_rows: 14389
max_text_length = 3200
min_summary_length = 30
# 0.25(max: 128/512) & 0.125(max: 64/512)
max_ratio = 0.25
min_ratio = 0.01
dota = load_from_disk("../data/reddit-dota-proc")
dota = dota.filter(lambda x: len(x["text"]) <= max_text_length)
dota = dota.filter(lambda x: len(x["summary"]) >= min_summary_length)
dota = dota.filter(lambda x: len(x["summary"])/len(x["text"]) <= max_ratio)
dota = dota.filter(lambda x: len(x["summary"])/len(x["text"]) >= min_ratio)
dota.flatten_indices().save_to_disk("../data/reddit-dota-qtr")
dota = load_from_disk("../data/reddit-dota-qtr")
dota